# Fast Predictive Distributions (LOVE)

This notebook will provide an overview of fast computation of variances and sampling using the LOVE algorithm, introduced [here](https://arxiv.org/abs/1803.06058).
Using LOVE can significantly reduce the computational costs of computing predictive distributions.
This can be especially useful in settings like small-cale Bayesian optimization, where predictions need to be made at a large number of candidate points, but there aren't enough training examples to warrant the use of sparse GP methods.

In [1]:
%matplotlib inline

In [2]:
import math

import torch
import gpytorch
import matplotlib.pyplot as plt
from tqdm import tnrange

## Loading Data

In this notebook we will train an exact GP on the `skillcraft` UCI dataset and compare the time required to make predictions with each model.
The code below will download a copy of the dataset that has been preprocessed (scaled and normalized).

In [3]:
import os
import urllib

DATA = os.path.join('data/skillcraft.mat')

if not os.path.isfile(DATA):  # .mat is a MatLab file
    print("Downloading \'skillcraft\' UCI dataset...")
    url = 'https://drive.google.com/uc?export=download&id=1xQ1vgx_bOsLDQ3RLbJwPxMyJHW7U9Eqd'
    urllib.request.urlretrieve(url, DATA)

For this example we'll simply split the dataset using the first 40% for training and the remaining 60% for testing.

In [4]:
from scipy.io import loadmat  # MatLab file loader


data = torch.Tensor(loadmat(DATA)['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

In [5]:
train_n = int(math.floor(0.4*len(X)))

train_x = X[:train_n, :].contiguous()  # .cuda()  # not using cuda here...
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

## Defining the GP Model

The model that we'll use here is essentially the same as in `gpytorch-regression.ipynb`. 

In [6]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel()
        )
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [7]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()  # once again, no .cuda()
model = GPRegressionModel(train_x, train_y, likelihood)

## Training the Model

In [8]:
model.train()
likelihood.train()

optimizer = torch.optim.Adam(
    [{'params': model.parameters()}],  # includes likelihood parameters
    lr=0.1,
)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)  # loss


def train(n):  # convenient for time tests
    pbar = tnrange(n)
    for i in pbar:
        optimizer.zero_grad()
        output = model(train_x)
        loss = -1 * mll(output, train_y)
        loss.backward()
        
        print(f"{i+1:3d}/{n:3d} | Loss: {loss.item(): .3f}")
        
        optimizer.step()
        

In [9]:
%time train(100)

  1/100 | Loss:  1.060
  2/100 | Loss:  0.995
  3/100 | Loss:  0.936
  4/100 | Loss:  0.878
  5/100 | Loss:  0.821
  6/100 | Loss:  0.773
  7/100 | Loss:  0.719
  8/100 | Loss:  0.672
  9/100 | Loss:  0.623
 10/100 | Loss:  0.579
 11/100 | Loss:  0.539
 12/100 | Loss:  0.493
 13/100 | Loss:  0.455
 14/100 | Loss:  0.413
 15/100 | Loss:  0.379
 16/100 | Loss:  0.344
 17/100 | Loss:  0.311
 18/100 | Loss:  0.278
 19/100 | Loss:  0.248
 20/100 | Loss:  0.221
 21/100 | Loss:  0.199
 22/100 | Loss:  0.174
 23/100 | Loss:  0.152
 24/100 | Loss:  0.137
 25/100 | Loss:  0.120
 26/100 | Loss:  0.105
 27/100 | Loss:  0.096
 28/100 | Loss:  0.087
 29/100 | Loss:  0.079
 30/100 | Loss:  0.078
 31/100 | Loss:  0.075
 32/100 | Loss:  0.076
 33/100 | Loss:  0.078
 34/100 | Loss:  0.079
 35/100 | Loss:  0.086
 36/100 | Loss:  0.081
 37/100 | Loss:  0.088
 38/100 | Loss:  0.085
 39/100 | Loss:  0.088
 40/100 | Loss:  0.088
 41/100 | Loss:  0.089
 42/100 | Loss:  0.089
 43/100 | Loss:  0.090
 44/100 | L

## Inference

In the cell below we will examine how long it takes to generate predictions using the standard SKI testing code, without any form of acceleration.

### Compute Exact Predicitons

In [10]:
import time

model.eval()  # evaluation mode for inference
likelihood.eval()

with torch.no_grad():  # standard SKI testing code
    start = time.time()
    preds = likelihood(model(test_x))
    exact_covar = preds.covariance_matrix
    exact_covar_time = time.time() - start

In [11]:
print(f'Time to compute exact mean and covariances: {exact_covar_time:.2f}secs')

Time to compute exact mean and covariances: 3.36secs


#### Cleanup

In order to get comparable timing results, we're going to perform some garbage cleanup with the code below.

In [12]:
import gc


def cleanup():
    gc.collect()
    torch.cuda.empty_cache()  # probably not needed since we're not using CUDA
    model.train()
    likelihood.train()

In [13]:
cleanup()

### Compute Predictions with LOVE

To compute predictions with LOVE, utilize the context manager `gpytorch.fast_pred_var()`.
You can also change the settings for LOVE with additional context managers.
For example, using `gpytorch.settings.max_root_decomposition_size(35)` affects the accuracy of the LOVE solver (passing larger values increases accuracy, but slows computation).

In [14]:
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.fast_pred_var(), gpytorch.settings.max_root_decomposition_size(25):
    start = time.time()
    preds_love = likelihood(model(test_x))
    fast_covar = preds_love.covariance_matrix
    fast_time = time.time() - start

In [15]:
print(f'Time to compute exact mean and covariances: {exact_covar_time:.2f}secs')
print(f'Time to compute mean and covariances with LOVE: {fast_time:.2f}secs')

Time to compute exact mean and covariances: 3.36secs
Time to compute mean and covariances with LOVE: 1.71secs


### Compute Predictions with LOVE using cached results

The above cell computed the caches required to get fast predicitons.
Once this is done we are able to use these cached results to make predictions _extremely_ fast (so long as we don't put our model back in training mode).

In [16]:
with torch.no_grad(), gpytorch.fast_pred_var(), gpytorch.settings.max_root_decomposition_size(25):
    start = time.time()
    preds_love_cache = likelihood(model(test_x))
    fast_covar_cache = preds_love_cache.covariance_matrix
    fast_time_cache = time.time() - start

In [17]:
print(f'Time to compute exact mean and covariances: {exact_covar_time:.2f}secs')
print(f'Time to compute mean and covariances with LOVE: {fast_time:.2f}secs')
print(f'Time to compute mean and covariances with LOVE using cache: {fast_time_cache:.2f}secs')

Time to compute exact mean and covariances: 3.36secs
Time to compute mean and covariances with LOVE: 1.71secs
Time to compute mean and covariances with LOVE using cache: 1.21secs


## Computing the Error Between Exact and Fast Values

In [18]:
mae = (exact_covar - fast_covar).abs().mean()
print(f'Mean Absolute Error between exact covar matrix and fast covar matrix: {mae:.10f}')

Mean Absolute Error between exact covar matrix and fast covar matrix: 0.0005233271
